In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
import time

def split_xy(df):
    y = df['Target']
    x = df.drop('Target',axis=1)
    return x,y

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('mode.chained_assignment', 'raise')SimpleImputerSimpleImputerSimpleImputer

In [4]:
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        
train = pd.read_csv('../suspicious-transaction-detection/train.csv')
# test = pd.read_csv('suspicious-transaction-detection/test.csv')

N_train, dim = train.shape
# N_test, _ = test.shape

# X_train = train.drop(['Target'], axis=1)
# y_train = train['Target'].copy()

In [ ]:
med_features = np.concatenate((['Amount'],
                               ['T_{}'.format(t) for t in range(15)], 
                               ['C_{}'.format(c) for c in range(9, 23)],
                               ['C_26', 'C_27', 'C_28'], 
                               ['V_{}'.format(v) for v in range(339)], 
                               ['O_0', 'O_1', 'O_5', 'O_6', 'O_9', 'O_18', 'O_31', 'O_36'],
                               ['A_0', 'M_1']
                              ))
mod_features = np.concatenate((['C_0', 'C_1', 'C_2', 'C_3', 'C_4', 'C_6', 'C_7', 'C_8', 'C_23'],
                               ['O_2', 'O_3', 'O_10', 'O_11', 'O_12', 'O_15', 'O_20', 'O_21', 'O_22', 'O_24', 'O_26', 'O_27', 'O_28', 'O_29', 'O_30', 'O_32', 'O_33', 'O_34', 'O_35', 'O_37', 'O_38'],
                               ['A_1']
                              ))
oh_features = np.concatenate((['Goods', 'C_5', 'C_24', 'C_25'],
                              ['O_4', 'O_7', 'O_8', 'browser', 'O_16', 'O_17', 'os', 'O_23', 'O_25',  'O_39'],
                              ['E_same', 'M_0']
                             ))

print(len(med_features) + len(mod_features) + len(oh_features))

In [ ]:
# features = pd.concat([X_train, test]).reset_index(drop=True)
features = train

features = features.drop(['TransactionID', 'Timestamp', 'O_14'], axis=1)

features['browser'] = features['O_13'].str.split(' ').str[0]
features['os'] = features['O_19'].str.split(' ').str[0]
features['E_0'] = features['E_0'].fillna('empty')
features['E_same'] = np.where(features['E_0'] == features['E_1'], 'T', 'F')

features = features.drop(['O_13', 'O_19', 'E_0', 'E_1'], axis=1)

for feature in med_features:
    med = train[feature].median()
    features[feature] = features[feature].fillna(med)
for feature in mod_features:
    mod = train[feature].mode()[0]
    #print('{}: {}'.format(feature, mod))
    features[feature] = features[feature].fillna(mod)

features_prepared = pd.get_dummies(features, columns=oh_features)

features_prepared.replace('T', 1, inplace=True)
features_prepared.replace('F', 0, inplace=True)

# xTr = 

# X_train_prepared = features_prepared.iloc[:N_train, :].copy()
# X_test_prepared = features_prepared.iloc[N_train:, :].copy()

# print(X_train_prepared.shape)
# print(X_test_prepared.shape)

In [ ]:
features

In [ ]:

# WORK HERE

# X_train = train.drop(['Target'], axis=1)
# y_train = train['Target'].copy()

validation_data = features.sample(frac=.1)
pre_train_data = features.drop(validation_data).reset_index(drop=True)
xVa, yVa = split_xy(validation_data)

for train_idx,test_idx in KFold(n_splits=10, random_state=1, shuffle=True).split(pre_train_data):
    train_data = pre_train_data.iloc[train_idx]
    test_data = pre_train_data.iloc[test_idx]
    print(train_data)
    
    xTr, yTr = split_xy(train_data)
    xTe, yTe = split_xy(test_data)
    
    
    

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

forest = RandomForestClassifier()
# forest_error = cross_val_score(forest, X_train_prepared, y_train, cv=5)
# print(forest_error.mean())
# print(forest_error.std())

forest.fit(X_train_prepared, y_train)
y_pred = forest.predict_proba(X_test_prepared)
roc = roc_auc_score(y_test,y_pred)
# my_submission = pd.DataFrame({'TransactionID': test.TransactionID, 'Target': y_pred[:, 1]})
# my_submission.to_csv('submission.csv', index=False)

In [ ]:
X_test_prepared

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,roc_auc_score

In [ ]:
grid = {}
grid['knn'] = {'p':[1,2,3], 'n_neighbors':[1,2,3,4,5,6,7,8,9]}

grid['rf'] = {'criterion': ['gini','entropy'], 'n_estimators': [50, 100], 'max_depth':[None,2,4,6], 'min_samples_split':[2,3,4], 'min_samples_leaf':[1,2], 'max_features':[None,'auto']}

grid['svmp'] = {'kernel':['poly'],'degree':[2,3], 'gamma':['scale'], 'coef0':[.01,.1,1,5,10,20,30], 'C':[.01,.1,.5,1], 'tol':[1e-1,1e-2,1e-3,1e-4,1e-5,1e-6]}
grid['svmg'] = {'kernel':['rbf'],'gamma':['auto'],'C':[.01,.1,.5,1], 'tol':[1e-1,1e-2,1e-3,1e-4,1e-5,1e-6]}

# grid['nnrelu'] = {'hidden_layer_sizes':[(10,30,10),(10,10,10)],'activation':["relu"],   'learning_rate':['constant','adaptive'],'max_iter':[100,500,800],'alpha':[.1,.01,.001,.0001,.00001],'tol':[1e-2,1e-3,1e-4,1e-5],'beta_1':[.01,.1,.25,.5,.7,.9],'beta_2':[.015,.1,.25,.4,.65,.9],'epsilon':[1e-4,1e-5,1e-6,1e-7]}
# grid['nnsig'] = {'hidden_layer_sizes':[(10,30,10),(10,10,10)],'activation':["logistic"],'learning_rate':['constant','adaptive'],'max_iter':[100,500,800],'learning_rate_init':[.01,.001,.0001],'alpha':[.1,.01,.001,.0001,.00001],'tol':[1e-2,1e-3,1e-4,1e-5],'beta_1':[.01,.1,.25,.5,.7,.9],'beta_2':[.015,.1,.25,.4,.65,.9],'epsilon':[1e-4,1e-5,1e-6,1e-7]}

In [ ]:
clf_knn = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=grid['knn'], n_jobs=-1, cv=5)
clf_knn.fit(X_train_prepared, y_train)
preds_knn = clf_knn.predict_proba(X_test_prepared)

clf_rf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=grid['rf'], n_jobs=-1, cv=5)
clf_rf.fit(X_train_prepared, y_train)
preds_rf = clf_rf.predict_proba(X_test_prepared)

clf_svmp = GridSearchCV(estimator=svm.SVC(), param_grid=grid['svmp'], n_jobs=-1, cv=5)
clf_svmp.fit(X_train_prepared, y_train)
preds_svmp = clf_svmp.predict_proba(X_test_prepared)

clf_svmg = GridSearchCV(estimator=svm.SVC(), param_grid=grid['svmg'], n_jobs=-1, cv=5)
clf_svmg.fit(X_train_prepared, y_train)
preds_svmg = clf_svmg.predict_proba(X_test_prepared)
